In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import epiweeks

In [2]:
# Initialize Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Drew\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
# Visit gender data source url
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTLO0Kf-G8sP1If_Fhj5GapXHSchZ3OwLJmGq6fF2h9-gE8rHrNxyhD_nm4Oa4YicmhtZ3MLcBoQxbH/pubhtml?wmode=opaque#"
browser.visit(url)

time.sleep(1)

In [4]:
# Initialize column list
columns = ['location','date','cases_total','male_cases',
           'female_cases','percent_male_cases',
           'percent_female_cases','male_case_rate',
           'female_case_rate','deaths_total','male_deaths',
           'female_deaths','percent_male_deaths',
           'percent_female_deaths','male_death_rate',
           'female_death_rate']

In [7]:
# Scrape site and return a dataframe with the displayed week's data
def scrape_weekly_data():
    
    # Scrap into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    displayed_table = soup.find('div', style='position: relative;')
    
    # Initialize week's dataframe
    week_df = pd.DataFrame(columns=columns)
    
    # Traverse rows 5-57 containing data cells
    for row_index in range(5,58):
        row = displayed_table.find('div', text=row_index).parent.parent
        
        # Get data from relevant cells
        cells = row.find_all('td')[0:16]
        
        # Add row to week table list
        week_df.loc[len(week_df)] = cells
    
    return week_df

In [8]:
week_buttons = browser.find_by_tag('ul').first.find_by_tag('li')

# Create blank csv file with headers
init_df = pd.DataFrame(columns=columns)
init_df.to_csv('gender_data_raw.csv', index=False)

for button in week_buttons[0:2]:
    button.click()
    #week_df_list = week_df_list + scrape_weekly_data()
    week_df = scrape_weekly_data()
    week_df.to_csv('gender_data_raw.csv', mode='a', index=False, header=False)